In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import re
import browser_cookie3
from dateutil import parser
import time

In [19]:
def reviewUrl(asin,cookie,header):
    try:
        url = f'https://www.amazon.in/dp/{asin}'
        page = requests.get(url,cookies=cookie,headers=header)
        if page.status_code==200:
            soup=BeautifulSoup(page.content)
            review_url = 'https://www.amazon.in'+soup.find("a",{'data-hook':"see-all-reviews-link-foot"})['href']
            return review_url
        else:
            print(f"Unable to access the {url} with error code {page.status_code}")
            return ''
    except:
        raise Exception


def getlastPage(review_url):
    page = requests.get(review_url,cookies=cookie,headers=header)
    soup=BeautifulSoup(page.content)
    rating_count  = re.sub('\D', '', soup.find("span",{"class":"a-size-base a-color-secondary"}).text)
    review_rating = soup.find("div",{"class":"a-row a-spacing-base a-size-base"}).text.replace('\n','').replace(',','').strip()
    review_rating_lst = [i for i in review_rating.split() if i.isdigit()]
    if len(review_rating_lst) == 2:
        review_count = int(review_rating_lst[1])
        if review_count <=20:
            totalPages = 1
        else:
            totalPages = round(review_count/20)+1
    else:
        print(f'Please check if the product have any reviews {review_url}')
    return totalPages+1


In [13]:
def reviewData(rvw):
    try:
        profile_name = rvw.find("span",{"class":"a-profile-name"}).text
    except:
        profile_name = ''

    try:
        rd = rvw.find("span",{"data-hook":"review-date"}).text
        review_date = str(parser.parse(rd.split(' on ')[1]))
        review_country = rd.split(' on ')[0].split()[-1]
    except:
        review_date = ''
        review_country = ''
    try:
        review_title = rvw.find("a",{"data-hook":"review-title"}).text.strip()
    except:
        review_title = ''

    try:
        verified_purchase = rvw.find("span",{"data-hook":"avp-badge"}).text.strip()
    except:
        verified_purchase = ''

    try:
        review_body = rvw.find("span",{"data-hook":"review-body"}).text.strip()
    except:
        review_body = ''

    try:
        review_helpful = rvw.find("span",{"data-hook":"helpful-vote-statement"}).text.strip()
    except:
        review_helpful = ''

    try:
        user_rating = rvw.find("i",{"data-hook":"review-star-rating"}).text.split()[0]
    except:
        user_rating = ''
    return {'profile_name':profile_name,'review_date':review_date,'review_country':review_country,'review_title':review_title,'verified_purchase':verified_purchase,'review_body':review_body,'review_helpful':review_helpful,'user_rating':user_rating}

In [14]:
asin = 'B08HV83HL3'
cookie = browser_cookie3.chrome(domain_name='.amazon.in')
header = {
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.122 Safari/537.36", 
    "Accept-Encoding":"gzip, deflate", 
    "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", 
    "DNT":"1",
    "Connection":"close", 
    "Upgrade-Insecure-Requests":"1"
}

In [15]:

review_url = reviewUrl(asin,cookie,header)

if len(review_url) > 0:
    totalPages = getlastPage(review_url)
    review_df = pd.DataFrame(columns=['profile_name','review_date','review_country','review_title','verified_purchase',
                                      'review_body','review_helpful','user_rating'])
    for i in range(1,totalPages):
        if i%50 ==0:
            time.sleep(5)
            print(f'Processing {i} page out of {totalPages}')
            
        url = f'{review_url}&pageSize=20&pageNumber={i}'
        cookie = browser_cookie3.chrome(domain_name='.amazon.in')
        page = requests.get(url,cookies=cookie,headers=header)
        if page.status_code==200:
            soup=BeautifulSoup(page.content)
            try:
                review_list =soup.findAll("div",{"data-hook":"review"})
                for rvw in review_list:
                    review_df = review_df.append(pd.DataFrame([reviewData(rvw)]),ignore_index=True)
                
            except:
                raise Exception
        else:
            print('f"Unable to access the page {i} with error code {page.status_code}"')
    
    print(f'{soup.find("a",{"data-hook":"product-link"}).text}\n')
    print(f'Rating is : {soup.find("span",{"data-hook":"rating-out-of-text"}).text}')
    if not os.path.exists('data'):
        os.makedirs('data')
    review_df.to_csv(f'data/{asin}.csv',index=False)

Processing 50 page out of 2062
Processing 100 page out of 2062
Processing 150 page out of 2062
Processing 200 page out of 2062
Processing 250 page out of 2062
Processing 300 page out of 2062
Processing 350 page out of 2062
Processing 400 page out of 2062
Processing 450 page out of 2062
Processing 500 page out of 2062
Processing 550 page out of 2062
Processing 600 page out of 2062
Processing 650 page out of 2062
Processing 700 page out of 2062
Processing 750 page out of 2062
Processing 800 page out of 2062
Processing 850 page out of 2062
Processing 900 page out of 2062
Processing 950 page out of 2062
Processing 1000 page out of 2062
Processing 1050 page out of 2062
Processing 1100 page out of 2062
Processing 1150 page out of 2062
Processing 1200 page out of 2062
Processing 1250 page out of 2062
Processing 1300 page out of 2062
Processing 1350 page out of 2062
Processing 1400 page out of 2062
Processing 1450 page out of 2062
Processing 1500 page out of 2062
Processing 1550 page out of 20

In [18]:
review_df.shape

(10000, 8)

In [22]:
review_df

,profile_name,review_date,review_country,review_title,verified_purchase,review_body,review_helpful,user_rating
0,Baskaran Rathinam,2020-08-14 00:00:00,India,Too slow,Verified Purchase,It takes 40 to 50 mins to charge my mobile pho...,"1,205 people found this helpful",1.0
1,Manish,2020-03-20 00:00:00,India,Poor charging capacity,Verified Purchase,"Power bank has only 1 purpose, to charge the p...",814 people found this helpful,1.0
2,Sandeep Bhattacharya,2019-12-11 00:00:00,India,The best battery bank in town,Verified Purchase,Do not think twice when you need a high power ...,861 people found this helpful,5.0
3,Biju,2020-10-29 00:00:00,India,Charging Speed!!,Verified Purchase,"This is an amazing product, Tom Cruise deliver...",356 people found this helpful,1.0
4,Manu Dileep,2020-09-27 00:00:00,India,Mi Power Bank 20000 Mah,Verified Purchase,Only First impressions now( First I don't tru...,196 people found this helpful,1.0
...,...,...,...,...,...,...,...,...
9995,Dev Patel,2021-06-09 00:00:00,India,worst power bank ever product are not sent as ...,Verified Purchase,i don't like this producte,,1.0
9996,risibble-ing,2020-01-25 00:00:00,India,Superb product,Verified Purchase,"Superb product, good quality material, long la...",,4.0
9997,Inndra,2020-08-28 00:00:00,India,Awesome,Verified Purchase,Awesome,,5.0
9998,Sundhararajan,2021-09-24 00:00:00,India,Tirupur,Verified Purchase,Power Bank fast playback,,2.0


In [23]:
review_df['verified_purchase'].value_counts(dropna=False,normalize=True)

Verified Purchase    1.0
Name: verified_purchase, dtype: float64

In [24]:
review_df['review_title'].value_counts(dropna=False)

Good                                                    598
Good product                                            267
Nice                                                    161
Value for money                                         120
Nice product                                            111
                                                       ... 
Cannot think of better than this. Go for it               1
USB cable provided is not type C. Its old micro USB.      1
Dont just go with the colour                              1
Must buy 👍👍                                               1
Tirupur                                                   1
Name: review_title, Length: 6263, dtype: int64